In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
from random import sample

In [2]:
df = pd.read_csv('swift.csv')

In [3]:
df_copy = df.copy()

In [4]:
size = 1000000

In [5]:
df = df_copy.copy()[:size]

In [6]:
df.isna().any()

Transaction Ref    False
Originator         False
Beneficiary        False
Type               False
Currency           False
Value              False
Flag               False
Fulldate           False
Intermediary       False
dtype: bool

In [7]:
%%time
df = df_copy.copy()[:size]
df['Fulldate'] = pd.to_datetime(df['Fulldate'])

df['OriginatorCountry'] = df['Originator'].str.strip().str[4:6]
df['IntermediaryCountry'] = df['Intermediary'].str.strip().str[4:6]
df['BeneficiaryCountry'] = df['Originator'].str.strip().str[4:6]

df['OriginatorCountry_BeneficiaryCountry'] = df['OriginatorCountry']+df['BeneficiaryCountry']
df['OriginatorCountry_IntermediaryCountry'] = df['OriginatorCountry']+df['IntermediaryCountry']
df['IntermediaryCountry_BeneficiaryCountry'] = df['IntermediaryCountry']+df['BeneficiaryCountry']
df['OriginatorCountry_IntermediaryCountry_BeneficiaryCountry'] = df['OriginatorCountry']+df['IntermediaryCountry']+df['BeneficiaryCountry']

Wall time: 2.07 s


# Features Client

In [8]:
def getLatency(x,name_col):
    return (x[name_col]-x[name_col].shift(1)).dt.total_seconds().fillna(0)

In [9]:
def get_circuit(x):
    return sum(df.loc[x.index,'Beneficiary']==df.loc[x.index,'Intermediary'])

In [10]:
def get_circuit(x):
    return sum(x.str[:2]!=x.str[2:4])

In [11]:
def base_features_client(type_client,period,relation):
    
    cols_start = df.columns
    
    groupby = []
        
    if period == '10Days':
        groupby = type_client+[pd.Grouper(key='Fulldate', axis=0, freq='10D')]
    elif period == 'Month':
        groupby = type_client+[pd.Grouper(key='Fulldate', axis=0, freq='30D')]
    elif period == 'Global':
        groupby = type_client

    name = period+'_'+'_'.join(type_client)
    
    print(groupby)
    #FREQUENCY
    #print('frequency')
    
    df['frequency_'+name] = df.groupby(groupby)['Value'].transform('count')
    
    #AMOUNT
    #print('amount')
    
    df['sum_value_'+name] = df.groupby(groupby)['Value'].transform('sum')
    df['max_value_'+name] = df.groupby(groupby)['Value'].transform('max')
    df['min_value_'+name] = df.groupby(groupby)['Value'].transform('min')
    df['avg_value_'+name] = df.groupby(groupby)['Value'].transform('sum')/df['frequency_'+name]
    
    if relation == False:
        # LATENCY
        #print('latency')
        df['latency_'+name] = df.groupby(groupby, as_index=False).apply(getLatency,name_col='Fulldate').reset_index(level=0, drop=True)


        # NB RELATION
        #print('nb relation')
        if type_client[0] == 'Originator':
            df['number_relation_'+name] = df.groupby(['Originator'])['Beneficiary'].transform('nunique')
        elif type_client[0] =='Beneficiary':
            df['number_relation_'+name] = df.groupby(['Beneficiary'])['Originator'].transform('nunique')
        
    # NB OF DISTINCT CURRENCY
    #print('currency 1')
    
    df['number_distinct_currency_'+name] = df.groupby(['Originator'])['Currency'].transform('nunique')
    
    # NB WITH CURRENT CURRENCY
    #print('currency 2')
        
    df['frequency_with_currency_'+name] = df.groupby(groupby+['Currency'])['Value'].transform('count')
    
    if relation == False:
    
        # NB OF DISTINCT COUNTRIES
        #print('country 1')

        if type_client[0] == 'Originator' :
            df['number_distinct_country_'+name] = df.groupby(['Originator'])['BeneficiaryCountry'].transform('nunique')
        elif type_client[0] =='Beneficiary':
            df['number_distinct_country_'+name] = df.groupby(['Beneficiary'])['OriginatorCountry'].transform('nunique')

        # NB WITH CURRENT COUNTRY
       # print('country 2')

        if type_client[0] == 'Originator':
            df['frequency_to_Beneficiary_Country_'+name] = df.groupby(groupby+['BeneficiaryCountry'])['Value'].transform('count')

        elif type_client[0] =='Beneficiary':
            df['frequency_to_Originator_Country_'+name] = df.groupby(groupby+['OriginatorCountry'])['Value'].transform('count')


        # NB WITH INTERMEDIATE
        #print('intermediary 1')

        if type_client[0] == 'Originator':
            df['number_with_intermediary_'+name] = df.groupby(['Originator'])['IntermediaryCountry_BeneficiaryCountry'].transform(get_circuit)
        elif type_client[0] =='Beneficiary' :
            df['number_with_intermediary_'+name] = df.groupby(['Beneficiary'])['IntermediaryCountry_BeneficiaryCountry'].transform(get_circuit)

        # NB DISTINCT CIRCRUIT
        #print('intermediary 2')

        if type_client[0] == 'Originator':
            df['number_dinstinct_circuit_with_intermediary_'+name] = df.groupby(['Originator'])['IntermediaryCountry_BeneficiaryCountry'].transform('nunique')
        elif type_client[0] =='Beneficiary':
            df['number_dinstinct_circuit_with_intermediary_'+name] = df.groupby(['Beneficiary'])['OriginatorCountry_IntermediaryCountry'].transform('nunique')

        # NB CURRENT CIRCRUIT
        #print('intermediary 3')

        if type_client[0] == 'Originator':
            df['number_current_circuit_with_intermediary_'+name] = df.groupby(['Originator','IntermediaryCountry_BeneficiaryCountry'])['Value'].transform('count')
        elif type_client[0] =='Beneficiary':
            df['number_current_circuit_with_intermediary_'+name] = df.groupby(['Beneficiary','OriginatorCountry_IntermediaryCountry'])['Value'].transform('count')

    
    cols_end = df.columns
    
    new_cols = list(set(cols_end) - set(cols_start))
    
    return new_cols

In [12]:
%%time 

array = []
name_configuration = []

array.append(base_features_client(['Beneficiary'],'Global',False))
array.append(base_features_client(['Beneficiary'],'10Days',False))
array.append(base_features_client(['Beneficiary'],'Month',False))
name_configuration.append('Beneficiary-Global')
name_configuration.append('Beneficiary-10Days')
name_configuration.append('Beneficiary-Month')

array.append(base_features_client(['Originator'],'Global',False))
array.append(base_features_client(['Originator'],'10Days',False))
array.append(base_features_client(['Originator'],'Month',False))
name_configuration.append('Originator-Global')
name_configuration.append('Originator-10Days')
name_configuration.append('Originator-Month')

array.append(base_features_client(['Intermediary'],'Global',False))
array.append(base_features_client(['Intermediary'],'10Days',False))
array.append(base_features_client(['Intermediary'],'Month',False))
name_configuration.append('Intermediary-Global')
name_configuration.append('Intermediary-10Days')
name_configuration.append('Intermediary-Month')

array.append(base_features_client(['Originator','Beneficiary'],'Global',True))
array.append(base_features_client(['Originator','Beneficiary'],'10Days',True))
array.append(base_features_client(['Originator','Beneficiary'],'Month',True))
name_configuration.append('Originator-Beneficiary-Global')
name_configuration.append('Originator-Beneficiary-10Days')
name_configuration.append('Originator-Beneficiary-Month')

array.append(base_features_client(['Originator','Intermediary'],'Global',True))
array.append(base_features_client(['Originator','Intermediary'],'10Days',True))
array.append(base_features_client(['Originator','Intermediary'],'Month',True))
name_configuration.append('Originator-Intermediary-Global')
name_configuration.append('Originator-Intermediary-10Days')
name_configuration.append('Originator-Intermediary-Month')

array.append(base_features_client(['Intermediary','Beneficiary'],'Global',True))
array.append(base_features_client(['Intermediary','Beneficiary'],'10Days',True))
array.append(base_features_client(['Intermediary','Beneficiary'],'Month',True))
name_configuration.append('Intermediary-Beneficiary-Global')
name_configuration.append('Intermediary-Beneficiary-10Days')
name_configuration.append('Intermediary-Beneficiary-Month')

['Beneficiary']
['Beneficiary', TimeGrouper(key='Fulldate', freq=<10 * Days>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', base=0)]
['Beneficiary', TimeGrouper(key='Fulldate', freq=<30 * Days>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', base=0)]
['Originator']
['Originator', TimeGrouper(key='Fulldate', freq=<10 * Days>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', base=0)]
['Originator', TimeGrouper(key='Fulldate', freq=<30 * Days>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', base=0)]
['Intermediary']
['Intermediary', TimeGrouper(key='Fulldate', freq=<10 * Days>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', base=0)]
['Intermediary', TimeGrouper(key='Fulldate', freq=<30 * Days>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', base=0)]
['Originator', 'Beneficiary']
['Originator', 'Beneficiary', TimeGroupe

# Features Category

In [13]:
def base_features_transaction(column,period):
    
    cols_start = df.columns
        
    if period == '10Days':
        groupby = [column,pd.Grouper(key=name_col_date, axis=0, freq='10D')]
    elif period == 'Month':
        groupby = [column,pd.Grouper(key=name_col_date, axis=0, freq='30D')]
    elif period == 'Global':
        groupby = column

    name = period+'_'+'_'.join(column)
    
    #FREQUENCY
    #print('frequency')
    
    df['frequency_'+name] = df.groupby(groupby)['Value'].transform('count')
    
    #AMOUNT
    #print('amount')
    
    df['sum_value_'+name] = df.groupby(groupby)['Value'].transform('sum')
    df['max_value_'+name] = df.groupby(groupby)['Value'].transform('max')
    df['min_value_'+name] = df.groupby(groupby)['Value'].transform('min')
    df['avg_value_'+name] = df.groupby(groupby)['Value'].transform('sum')/df['frequency_'+name]
    
    # LATENCY
    #print('latency')
    df['latency_'+name] = df.groupby(groupby, as_index=False).apply(getLatency,name_col='Fulldate').reset_index(level=0, drop=True)
    
    # NB RELATION
    #print('nb relation')
    df['number_relation_originator_'+name] = df.groupby(groupby)['Originator'].transform('nunique')
    df['number_relation_beneficiary_'+name] = df.groupby(groupby)['Beneficiary'].transform('nunique')
    df['number_relation_intermediary_'+name] = df.groupby(groupby)['Intermediary'].transform('nunique')
        
    # NB OF DISTINCT CURRENCY
    #print('currency ')
    
    if column != 'Currency':
        df['number_distinct_currency_'+name] = df.groupby(groupby)['Currency'].transform('nunique')
        df['frequency_with_currency_'+name] = df.groupby(groupby+['Currency'])['Value'].transform('count')
    
    
    # NB OF DISTINCT COUNTRIES
    #print('country 1')
    
    if column not in ['OriginatorCountry','BeneficiaryCountry','IntermediaryCountry']:
        df['number_distinct_originator_country_'+name] = df.groupby(groupby)['OriginatorCountry'].transform('nunique')
        df['number_distinct_beneficiary_country_'+name] = df.groupby(groupby)['BeneficiaryCountry'].transform('nunique')
        df['number_distinct_intermediary_country_'+name] = df.groupby(groupby)['IntermediaryCountry'].transform('nunique')
        
        df['number_current_originator_country_'+name] = df.groupby(groupby)['OriginatorCountry'].transform('count')
        df['number_current_beneficiary_country_'+name] = df.groupby(groupby)['BeneficiaryCountry'].transform('count')
        df['number_current_intermediary_country_'+name] = df.groupby(groupby)['IntermediaryCountry'].transform('count')
            
    
    # NB WITH INTERMEDIATE
    #print('intermediary 1')
    df['number_with_intermediary_'+name] = df.groupby(groupby)['IntermediaryCountry_BeneficiaryCountry'].transform(get_circuit)
    
    # NB DISTINCT CIRCRUIT
    #print('intermediary 2')
    
    df['number_dinstinct_circuit_with_intermediary_'+name] = df.groupby(groupby)['IntermediaryCountry_BeneficiaryCountry'].transform('nunique')
        
    # NB CURRENT CIRCRUIT
    #print('intermediary 3')
    
    df['number_current_circuit_intermediaryCountry_beneficiaryCountry_'+name] = df.groupby(groupby+['IntermediaryCountry_BeneficiaryCountry'])['Value'].transform('count')
    df['number_current_circuit_originatorCountry_intermediaryCountry_'+name] = df.groupby(groupby+['OriginatorCountry_IntermediaryCountry'])['Value'].transform('count')
    df['number_current_circuit_originatorCountry_beneficiaryCountry_'+name] = df.groupby(groupby+['OriginatorCountry_BeneficiaryCountry'])['Value'].transform('count')
    df['number_current_circuit_originatorCountry_intermediaryCountry_beneficiaryCountry_'+name] = df.groupby(groupby+['OriginatorCountry_IntermediaryCountry_BeneficiaryCountry'])['Value'].transform('count')
    
    cols_end = df.columns
    
    new_cols = list(set(cols_end) - set(cols_start))
    
    return new_cols
        

In [14]:
%%time


base_feature = []
array.append(base_features_transaction(['Currency'],'Global'))
name_configuration.append('Currency-Global')
array.append(base_features_transaction(['OriginatorCountry'],'Global'))
name_configuration.append('OriginatorCountry-Global')
array.append(base_features_transaction(['BeneficiaryCountry'],'Global'))
name_configuration.append('BeneficiaryCountry-Global')
array.append(base_features_transaction(['IntermediaryCountry'],'Global'))
name_configuration.append('IntermediaryCountry-Global')
array.append(base_features_transaction(['Type'],'Global'))
name_configuration.append('Type-Global')


Wall time: 1min 46s


# Transaction Features

In [15]:


base_feature.extend(list(df_copy.select_dtypes(include=np.number).columns))
cols_start = df.columns

df['Hour'] = df['Fulldate'].dt.hour
df['Week_of_year'] = df['Fulldate'].dt.week
df['day_of_week'] = df['Fulldate'].dt.dayofweek
df['day_of_year'] = df['Fulldate'].dt.dayofyear
df['quarter'] = df['Fulldate'].dt.quarter
df['year'] = df['Fulldate'].dt.year
df['month'] = df['Fulldate'].dt.month
df['day'] = df['Fulldate'].dt.day
df['is_month_start'] = df['Fulldate'].dt.is_month_start
df['is_month_start'] = df['is_month_start'].map({True: 1, False:0})
df['is_month_end'] = df['Fulldate'].dt.is_month_end
df['is_month_end'] = df['is_month_end'].map({True: 1, False:0})
cols_end = df.columns          
new_cols = list(set(cols_end) - set(cols_start))

base_feature.extend(new_cols)

In [16]:
df.isna().any()

Transaction Ref    False
Originator         False
Beneficiary        False
Type               False
Currency           False
                   ...  
year               False
month              False
day                False
is_month_start     False
is_month_end       False
Length: 312, dtype: bool

# Classification

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from itertools import combinations
from sklearn.metrics import matthews_corrcoef
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as npobject

In [18]:
X = df.select_dtypes(include=np.number)
y = df['Flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
%%time

model = DecisionTreeClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(matthews_corrcoef(y_test, y_pred))
report = classification_report(y_test,y_pred,output_dict=True )
CM = confusion_matrix(y_test, y_pred)


TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print('les fraudes : ')
print('bien trouvée TP : ',TP)
print('pas trouvée FN : ',FN) 
print('les normaux')
print('bien trouvée TN : ',TN)
print('pas trouvée FP : ',FP)
report

0.5855209376457683
les fraudes : 
bien trouvée TP :  462
pas trouvée FN :  240
les normaux
bien trouvée TN :  198878
pas trouvée FP :  420
Wall time: 4min 23s


{'False': {'precision': 0.9987946845589047,
  'recall': 0.9978926030366587,
  'f1-score': 0.9983434400224891,
  'support': 199298},
 'True': {'precision': 0.5238095238095238,
  'recall': 0.6581196581196581,
  'f1-score': 0.5833333333333334,
  'support': 702},
 'accuracy': 0.9967,
 'macro avg': {'precision': 0.7613021041842143,
  'recall': 0.8280061305781584,
  'f1-score': 0.7908383866779112,
  'support': 200000},
 'weighted avg': {'precision': 0.9971274866446745,
  'recall': 0.9967,
  'f1-score': 0.9968867545480102,
  'support': 200000}}

In [20]:
%%time

modelRF = RandomForestClassifier(n_estimators=30,bootstrap = True,max_features = 'sqrt')

modelRF.fit(X_train, y_train)

y_pred = modelRF.predict(X_test)
print(matthews_corrcoef(y_test, y_pred))
report = classification_report(y_test,y_pred,output_dict=True )
report
CM = confusion_matrix(y_test, y_pred)

TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print('les fraudes : ')
print('bien trouvée TP : ',TP)
print('pas trouvée FN : ',FN)
print('les normaux')
print('bien trouvée TN : ',TN)
print('pas trouvée FP : ',FP)
report

0.7355223224980376
les fraudes : 
bien trouvée TP :  417
pas trouvée FN :  285
les normaux
bien trouvée TN :  199258
pas trouvée FP :  40
Wall time: 4min 45s


{'False': {'precision': 0.9985717364177145,
  'recall': 0.9997992955273008,
  'f1-score': 0.999185138940079,
  'support': 199298},
 'True': {'precision': 0.912472647702407,
  'recall': 0.594017094017094,
  'f1-score': 0.7195858498705779,
  'support': 702},
 'accuracy': 0.998375,
 'macro avg': {'precision': 0.9555221920600607,
  'recall': 0.7969081947721974,
  'f1-score': 0.8593854944053285,
  'support': 200000},
 'weighted avg': {'precision': 0.9982695286163238,
  'recall': 0.998375,
  'f1-score': 0.998203745435445,
  'support': 200000}}

In [21]:
%%time

import xgboost as xgb


xgb_model = xgb.XGBClassifier(estimators=2,objective="binary:logistic")
xgb_model.fit(X_train, y_train)


y_pred = xgb_model.predict(X_test)

print(matthews_corrcoef(y_test, y_pred))

report = classification_report(y_test,y_pred,output_dict=True )
report
CM = confusion_matrix(y_test, y_pred)

TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print('les fraudes : ')
print('bien trouvée TP : ',TP)
print('pas trouvée FN : ',FN)
print('les normaux')
print('bien trouvée TN : ',TN)
print('pas trouvée FP : ',FP)
report


[10:20:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.7716291244570288
les fraudes : 
bien trouvée TP :  463
pas trouvée FN :  239
les normaux
bien trouvée TN :  199249
pas trouvée FP :  49
Wall time: 3min 18s


{'False': {'precision': 0.9988019329483477,
  'recall': 0.9997541370209435,
  'f1-score': 0.9992778081477284,
  'support': 199298},
 'True': {'precision': 0.904296875,
  'recall': 0.6595441595441596,
  'f1-score': 0.7627677100494235,
  'support': 702},
 'accuracy': 0.99856,
 'macro avg': {'precision': 0.9515494039741739,
  'recall': 0.8296491482825515,
  'f1-score': 0.881022759098576,
  'support': 200000},
 'weighted avg': {'precision': 0.998470220194949,
  'recall': 0.99856,
  'f1-score': 0.9984476577034033,
  'support': 200000}}

# Approche

In [22]:
%%time
y_pred_array = []
y_pred_array_f1_score = []
models = []
models_f1_score = []

tree_param = {'criterion':['gini','entropy'], 'class_weight':[{0: w} for w in [2,10]]+[None]}

for idx,cols in enumerate(name_configuration):
    gs = GridSearchCV(DecisionTreeClassifier(), tree_param,scoring='f1_micro', n_jobs=-1, cv=2)
    model = gs.fit(X_train[array[idx]+base_feature], y_train)
    # model = DecisionTreeClassifier(criterion = 'gini')
    model.fit(X_train[array[idx]+base_feature], y_train)
    models.append(model)
    
    y_pred = model.predict(X_test[array[idx]+base_feature])
    
    report = classification_report(y_test,y_pred,output_dict=True )

    models_f1_score.append(report['True']['f1-score'])
    y_pred_array.append(np.multiply(y_pred, 1))
    y_pred_array_f1_score.append(np.multiply(y_pred, round(report['True']['f1-score'],2)))
    
    print(name_configuration[idx], 'base & ',round(report['True']['precision'],2),'&',round(report['True']['recall'],2),'&',round(report['True']['f1-score'],2),'\\\ \hline')

Beneficiary-Global base &  0.51 & 0.61 & 0.55 \\ \hline
Beneficiary-10Days base &  0.47 & 0.57 & 0.51 \\ \hline
Beneficiary-Month base &  0.47 & 0.57 & 0.52 \\ \hline
Originator-Global base &  0.2 & 0.26 & 0.23 \\ \hline
Originator-10Days base &  0.22 & 0.3 & 0.26 \\ \hline
Originator-Month base &  0.21 & 0.28 & 0.24 \\ \hline
Intermediary-Global base &  0.36 & 0.43 & 0.39 \\ \hline
Intermediary-10Days base &  0.21 & 0.25 & 0.23 \\ \hline
Intermediary-Month base &  0.25 & 0.31 & 0.28 \\ \hline
Originator-Beneficiary-Global base &  0.53 & 0.62 & 0.58 \\ \hline
Originator-Beneficiary-10Days base &  0.32 & 0.41 & 0.36 \\ \hline
Originator-Beneficiary-Month base &  0.51 & 0.59 & 0.55 \\ \hline
Originator-Intermediary-Global base &  0.38 & 0.49 & 0.43 \\ \hline
Originator-Intermediary-10Days base &  0.25 & 0.33 & 0.29 \\ \hline
Originator-Intermediary-Month base &  0.3 & 0.38 & 0.33 \\ \hline
Intermediary-Beneficiary-Global base &  0.41 & 0.49 & 0.45 \\ \hline
Intermediary-Beneficiary-10Day

In [26]:
%%time

maxF1 = 0
maxMCC = 0
alphaMax = 0
threshold = 0
listf1 = []
listmcc= []
listIndex = []

n = np.sum(y_pred_array, 0)
xi =  np.sum(y_pred_array_f1_score, 0)
xiPI = np.prod(y_pred_array_f1_score, 0)

#y_pred_final = xi/n  #arithmetic 
#y_pred_final = n/1/xi #harmonic
#y_pred_final = npobject.power(xiPI,(1/n)) #geometric
y_pred_final = np.sqrt(1/n*(xi*xi))  #quadratic

y_pred_final[np.isnan(y_pred_final)] = 0.0
y_pred_final[np.isneginf (y_pred_final)] = 0.0
y_pred_final = np.round(y_pred_final,2)
y_pred_final = (y_pred_final - np.min(y_pred_final)) / (np.max(y_pred_final) - np.min(y_pred_final))

for i in np.arange(0,np.max(y_pred_final),0.01):
    y_pred = np.where(y_pred_final>=i, 1, 0)
    report = classification_report(y_test,y_pred,output_dict=True )
    listf1.append(report['True']['f1-score'])
    listIndex.append(i)
    listmcc.append(matthews_corrcoef(y_test, y_pred))

    # print( i,' : ',report['True'])
if np.max(listf1) > maxF1:
    maxF1 = np.max(listf1)
    threshold = listIndex[np.argmax(listf1)]
    maxMCC = listmcc[np.argmax(listf1)]
    
print('----------------')

y_pred = np.where(y_pred_final>=threshold, 1, 0)
report = classification_report(y_test,y_pred,output_dict=True )
print('f1 : ',report['macro avg']['f1-score'])
print('mcc : ',maxMCC)
print('max : ',np.max(y_pred_final))
print('gap : ',np.max(y_pred_final)/50)
print('best threshold : ',threshold)
print('----------------')
report



c:\users\hamza\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
c:\users\hamza\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()


----------------
f1 :  0.8923482396652576
mcc :  0.7900779565058965
max :  1.0
gap :  0.02
best threshold :  0.56
----------------
Wall time: 29.5 s


{'False': {'precision': 0.9989571007400574,
  'recall': 0.9996889080673163,
  'f1-score': 0.9993228704274945,
  'support': 199298},
 'True': {'precision': 0.8884892086330936,
  'recall': 0.7037037037037037,
  'f1-score': 0.7853736089030208,
  'support': 702},
 'accuracy': 0.99865,
 'macro avg': {'precision': 0.9437231546865754,
  'recall': 0.85169630588551,
  'f1-score': 0.8923482396652576,
  'support': 200000},
 'weighted avg': {'precision': 0.998569358438762,
  'recall': 0.99865,
  'f1-score': 0.9985719085195435,
  'support': 200000}}

In [27]:
y_pred_final[np.isnan(y_pred_final)] = 0.0
y_pred_final[np.isneginf (y_pred_final)] = 0.0
y_pred_final = np.round(y_pred_final,2)
y_pred_final = (y_pred_final - np.min(y_pred_final)) / (np.max(y_pred_final) - np.min(y_pred_final))

for i in np.arange(0,1.1,0.1):
    y_pred = np.where(y_pred_final>=i, 1, 0)
    report = classification_report(y_test,y_pred,output_dict=True )
    listf1.append(report['True']['f1-score'])
    print('( ',round(i,2),' , ',round(report['macro avg']['recall'],2),' )')

(  0.0  ,  0.5  )
(  0.1  ,  0.92  )
(  0.2  ,  0.92  )
(  0.3  ,  0.91  )
(  0.4  ,  0.89  )
(  0.5  ,  0.87  )
(  0.6  ,  0.83  )
(  0.7  ,  0.79  )
(  0.8  ,  0.7  )
(  0.9  ,  0.57  )
(  1.0  ,  0.52  )


# Scores 

In [28]:
arr_conf = []
for i in np.transpose(y_pred_array):
    arr_conf.append([name_configuration[j] for j,x in enumerate(i) if x>0])

In [54]:
new_df = df_copy.loc[X_test.index] 
new_df['Fraudulent Score'] = y_pred_final
new_df['Decision Trees'] = arr_conf
new_df = new_df.sort_values(by=['Fraudulent Score'], ascending=False)

In [217]:
arrayIndexFraudulent = list(new_df[new_df['Fraudulent Score']>0].index)
arrayIndexFraudulent = sample(arrayIndexFraudulent,5)
for i in arrayIndexFraudulent:
    row = new_df.loc[i]
    print(i,' & ',row['Originator'],' & ',row['Intermediary'],' & ',row['Beneficiary'],' & ',row['Value'],' & ',row['Fulldate'],' & ',row['Currency'],' & ',row['Fraudulent Score'],'\\\ \hline')

442624  &  AQFSTHXX  &  SCGGIDXX  &  SCGGIDXX  &  44599  &  2019-02-13 12:03:00  &  FA8  &  0.14 \\ \hline
494182  &  BDCKMLXX  &  ORBJMTXX  &  JNFQUYXX  &  1186919  &  2019-02-19 09:09:00  &  189  &  0.35 \\ \hline
60337  &  BFPSSXXX  &  IZJZQAXX  &  IZJZQAXX  &  1417680  &  2019-01-07 10:58:00  &  3CB  &  0.61 \\ \hline
66348  &  CGINUYXX  &  KITSUYXX  &  KITSUYXX  &  481758  &  2019-01-07 14:33:00  &  273  &  0.8 \\ \hline
192537  &  ATCUIDXX  &  KCXSVIXX  &  KCXSVIXX  &  201231  &  2019-01-18 14:18:00  &  28B  &  0.17 \\ \hline


In [218]:
for i in arrayIndexFraudulent:
    row = new_df.loc[i]
    print(i,' & ',row['Fraudulent Score'],' & ',row['Decision Trees'],'\\\ \hline')

442624  &  0.14  &  ['Intermediary-10Days'] \\ \hline
494182  &  0.35  &  ['Originator-Beneficiary-Global'] \\ \hline
60337  &  0.61  &  ['Beneficiary-Global', 'Beneficiary-Month', 'Originator-Global', 'Originator-10Days', 'Originator-Month', 'Intermediary-10Days', 'Intermediary-Beneficiary-Month', 'Currency-Global', 'OriginatorCountry-Global', 'BeneficiaryCountry-Global', 'IntermediaryCountry-Global', 'Type-Global'] \\ \hline
66348  &  0.8  &  ['Beneficiary-10Days', 'Beneficiary-Month', 'Originator-Global', 'Originator-10Days', 'Originator-Month', 'Intermediary-Global', 'Intermediary-10Days', 'Intermediary-Month', 'Originator-Beneficiary-Month', 'Originator-Intermediary-Global', 'Originator-Intermediary-10Days', 'Intermediary-Beneficiary-Global', 'Intermediary-Beneficiary-10Days', 'Intermediary-Beneficiary-Month'] \\ \hline
192537  &  0.17  &  ['Intermediary-Month'] \\ \hline


# Get Path

In [304]:
arrayIndexFraudulent[4]

192537

In [296]:
row = new_df.loc[arrayIndexFraudulent[4]]

In [297]:
index_most_fraudulent_transaction = row.name

In [298]:
row['Decision Trees']

['Intermediary-Month']

In [299]:
first_decision_tree = row['Decision Trees'][0]

In [300]:
first_decision_tree

'Intermediary-Month'

In [301]:
index_model = name_configuration.index(first_decision_tree)

In [302]:
models[index_model].predict([X_test[array[index_model]+base_feature].loc[index_most_fraudulent_transaction]])

array([ True])

In [303]:
estimator = models[index_model].best_estimator_

n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

X_test_model = X_test[array[index_model]+base_feature]
index_loc = list(X_test_model.index).index(index_most_fraudulent_transaction)
node_indicator = estimator.decision_path(X_test_model)

leave_id = estimator.apply(X_test_model)


# HERE IS WHAT YOU WANT
sample_id = index_loc
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:

    if leave_id[sample_id] == node_id:  # <-- changed != to ==
        #continue # <-- comment out
        print("leaf node {} reached, no decision here".format(leave_id[sample_id])) # <--

    else: # < -- added else to iterate through decision nodes
        if (X_test_model.iloc[[sample_id], feature[node_id]].iloc[0] <= threshold[node_id]):
            threshold_sign = "<="
        else:
            threshold_sign = ">"

        print("%s : %s %s %s"
              % (X_test_model.columns[feature[node_id]],
                  X_test_model.iloc[[sample_id], feature[node_id]].iloc[0],
                 threshold_sign,
                 threshold[node_id]))

Rules used to predict sample 192173: 
frequency_Month_Intermediary : 7 > 1.5
Value : 201231 <= 442023.5
number_distinct_currency_Month_Intermediary : 7 <= 9.5
Value : 201231 > 57777.5
max_value_Month_Intermediary : 201231 <= 1218307.5
latency_Month_Intermediary : 12780.0 > 1050.0
Value : 201231 > 153756.5
max_value_Month_Intermediary : 201231 <= 667993.0
max_value_Month_Intermediary : 201231 <= 667151.5
min_value_Month_Intermediary : 14827 <= 293071.5
day_of_year : 18 <= 31.5
frequency_Month_Intermediary : 7 <= 227.5
latency_Month_Intermediary : 12780.0 > 1650.0
max_value_Month_Intermediary : 201231 > 154859.5
day_of_year : 18 <= 22.5
Value : 201231 > 197373.0
Value : 201231 > 197741.5
max_value_Month_Intermediary : 201231 <= 462673.0
frequency_Month_Intermediary : 7 <= 79.5
frequency_Month_Intermediary : 7 <= 11.5
latency_Month_Intermediary : 12780.0 <= 69630.0
avg_value_Month_Intermediary : 80933.71428571429 <= 85308.53125
leaf node 1668 reached, no decision here
